<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/2000px-TensorFlowLogo.svg.png width=20% img>

# Predicting the Number of Cylinders With <font color='orange'>Tensorflow 2.0</font>

In [ ]:
import os
os.chdir('c:/users/nicolas/documents/data/thecarconnection')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
tf.keras.backend.set_floatx('float64')
from sklearn.preprocessing import LabelEncoder

#### Loading the data

In [ ]:
df = pd.read_csv([i for i in os.listdir() if i.startswith('cleaned')][0])

In [ ]:
cols = ['Passenger Capacity', 'Passenger Doors', 'Front Shoulder Room (in)',
        'Front Head Room (in)', 'Second Leg Room (in)', 'Front Leg Room (in)',
        'Second Shoulder Room (in)', 'Second Head Room (in)',
        'Height, Overall (in)', 'Wheelbase (in)', 'Width, Max w/o mirrors (in)',
        'Fuel Tank Capacity, Approx (gal)', 'EPA Fuel Economy Est - Hwy (MPG)',
        'EPA Fuel Economy Est - City (MPG)', 'Third Gear Ratio (:1)',
        'First Gear Ratio (:1)', 'Fourth Gear Ratio (:1)',
        'Second Gear Ratio (:1)', 'Front Brake Rotor Diam x Thickness (in)',
        'Rear Brake Rotor Diam x Thickness (in)',
        'Turning Diameter - Curb to Curb', 'Gears', 'Net Horsepower',
        'Net Horsepower RPM', 'Net Torque', 'Net Torque RPM', 'Cylinders',
        'Displacement (L)', 'Displacement (cc)', 'Rear Tire Width',
        'Front Tire Width', 'Rear Wheel Size', 'Front Wheel Size', 'Tire Ratio']

In [ ]:
df = df[cols].sample(frac=1)

#### Encoding the input

In [ ]:
df.head().iloc[:, :4].reset_index(drop=True)

In [ ]:
x = df.drop('Cylinders', axis=1)
y = df['Cylinders']

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

#### Transforming the data

In [ ]:
ms = MinMaxScaler()
x = ms.fit_transform(x)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y)

#### Creating an iterable

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((xtrain, ytrain)).shuffle(25).batch(24)
test_ds = tf.data.Dataset.from_tensor_slices((xtest, ytest)).batch(24)

#### Creating a neural net class

In [ ]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = Dense(64, activation='relu')
        self.d2 = Dense(128, activation='relu')
        self.d3 = Dense(256, activation='relu')
        self.d4 = Dense(7, activation='softmax')
        
    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)
        return x

In [ ]:
model = MyModel()

#### Creating loss and accuracy objects

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [ ]:
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_loss = tf.keras.metrics.Mean(name='test_loss')

In [ ]:
optimizer = tf.keras.optimizers.Adam()

#### Step functions

In [ ]:
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) 
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_acc(targets, predictions)

In [ ]:
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs)
    t_loss = loss_object(targets, predictions)
    
    test_loss(t_loss)
    test_acc(targets, predictions)

#### Training the model

In [ ]:
for epoch in range(5):
    for data, labels in train_ds:
        train_step(data, labels)
        
    for test_data, test_labels in test_ds:
        test_step(test_data, test_labels)
        
    template = 'Epoch {:d}, Loss: {:>6.4f}, Acc.: {:>5.2%}, '\
    'Test Loss: {:>6.4f}, Test Acc.: {:>5.2%}'

    print(template.format(epoch+1,
                        train_loss.result(),
                        train_acc.result(),
                        test_loss.result(),
                        test_acc.result()))
        
    train_loss.reset_states()
    train_acc.reset_states()
    
    test_loss.reset_states()
    test_acc.reset_states()